# Testing notebooks

In [1]:
msg = "Welcome to Jupyter notebooks"
print(msg)

Welcome to Jupyter notebooks


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Define the scope
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# Add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name(
    "path/to/your/credentials.json", scope
)

# Authorize the clientsheet
client = gspread.authorize(creds)

# Get the instance of the Spreadsheet
sheet = client.open("your_google_sheet_name")

# Get the first sheet of the Spreadsheet
sheet_instance = sheet.get_worksheet(0)

# Get all the records of the data
records_data = sheet_instance.get_all_records()

# Convert the json to dataframe
df = pd.DataFrame.from_dict(records_data)
print(df)

In [1]:
import gspread
import pandas as pd

# Ruta al archivo JSON de credenciales
cred_file = "/home/mikel/mikelinv-1fa650952440.json"

In [3]:
# Autenticación con Google Sheets
gc = gspread.service_account(filename=cred_file)

# Abrir el documento de Google Sheets por nombre
spreadsheet = gc.open("Tracker")

In [10]:
# Seleccionar la pestaña (tab) por nombre
worksheet = spreadsheet.worksheet("Master")

# Obtener todos los valores de la hoja
data = worksheet.get_all_records()

# Convertir a DataFrame de Pandas
df_raw = pd.DataFrame(data)

In [ ]:
df = df_raw.copy()
df = df.astype(str)
df["Price"] = df["Price"].str.replace(r"[€,]", "", regex=True)
# df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# df.info()

In [14]:
print(df.dtypes)

Code                  object
Name                  object
Status                object
Price                 object
Shares                object
Value                 object
Currency portfolio    object
Type1                 object
Type2                 object
Advanced Index        object
Expected Yield        object
Risk                  object
Acc/Dist              object
Management            object
Isin                  object
Maturity              object
Coupon payment        object
Comision              object
Spain                 object
Europe                object
US                    object
EM                    object
Int                   object
Delta index           object
dtype: object
